In [1]:
%matplotlib inline
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

In [2]:
def imshowCV(img, axis = False, show = True):
    if not axis:
        plt.axis('off')
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if(show):
        plt.show()
    
def imshow(img, axis = False, colorbar = False, show = True):
    if not axis:
        plt.axis('off')
    plt.imshow(img)
    if(colorbar):
        plt.colorbar()
    if(show):
        plt.show()
    
def torch2Img(img, show = True):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    disp_img = to_np(img)
    if len(disp_img.shape) == 4:
        disp_img = disp_img[0]
    disp_img = disp_img.transpose((1,2,0))
    disp_img = disp_img * std + mean
    return disp_img
    
def imshowTorch(img, axis = False, show = True):
    if not axis:
        plt.axis('off')
    disp_img = torch2Img(img)
    plt.imshow(disp_img.astype(np.uint8))
    if(show):
        plt.show()

def plotImageScatter(img, choose, show = True):
    coords = np.unravel_index(choose, img.shape[:2])    
    plt.axis('off')
    plt.imshow(img.astype(np.uint8))    
    plt.scatter(coords[1], coords[0], 50)
    #plt.colorbar()
    if(show):
        plt.show()

In [3]:
dataset_root = '/home/bokorn/data/ycb/debug/'
object_id = 14
mode = "train"

In [4]:
from object_pose_utils.datasets.ycb_dataset import YcbDataset as YCBDataset
from object_pose_utils.datasets.image_processing import ImageNormalizer
from object_pose_utils.datasets.pose_dataset import OutputTypes as otypes

output_format = [otypes.OBJECT_LABEL,
                 otypes.QUATERNION, 
                 otypes.IMAGE_CROPPED, 
                 otypes.DEPTH_POINTS_MASKED_AND_INDEXES]

ycb_dataset = YCBDataset(dataset_root, mode=mode, 
                         object_list = [object_id], 
                         output_data = output_format, 
                         postprocessors = [ImageNormalizer()],
                         image_size = [640, 480], num_points=1000)

In [30]:
from object_pose_utils.datasets.ycb_video_dataset import YcbVideoDataset as YCBVideoDataset

dataset = YCBVideoDataset(ycb_dataset, 
                          interval = 4, 
                          video_len = 3)

dataset.setVideoId('0000')


In [31]:
print(dataset.getObjectIds())
print(dataset.getVideoIds())
print(len(dataset.index_list))


[14]
['0000']
9


In [32]:
idx = 0
data = dataset.getItem(idx)
trans = dataset.getCameraTransforms(idx)

In [33]:
from object_pose_utils.utils.multi_view_utils import applyTransform, computeCameraTransform

quats = []
for mat, d in zip(trans, data):
    quats.append(d[1])


In [34]:
quats

[tensor([ 0.7850, -0.3913,  0.2590,  0.4044]),
 tensor([ 0.7841, -0.3918,  0.2614,  0.4041]),
 tensor([ 0.7843, -0.3931,  0.2619,  0.4021])]

In [35]:
quats_trans = applyTransform(quats, trans)

[array([ 0.78501582, -0.39128241,  0.25902703,  0.40441743]),
 array([ 0.78503512, -0.39129683,  0.25902578,  0.40436679]),
 array([ 0.78505671, -0.39129698,  0.25897794,  0.40435576])]

In [37]:
from object_pose_utils.utils.pose_processing import quatAngularDiff
q0 = to_np(data[0][1])
for q, q_t in zip(quats, quat_trans):
    quatAngularDiff(q0, q)*180/np.pi
    print(, )


0.020641618571863796
0.2910394930802486
0.48652849477105786
